In [28]:
import pandas as pd
import json
import collections

In [7]:
CATS_gens = pd.read_csv("GIS/CATS_gens.csv")
pd.set_option('display.max_columns', None)

In [11]:
CATS_gens = CATS_gens.drop_duplicates(subset=['PlantCode'])

In [12]:
CATS_gens.head(20)

,PlantCode,GenID,bus,FuelType,Pg,Pmax,Pmin,Qg,Qmax,Qmin,Lat,Lon
0,34,1P,745,Conventional Hydroelectric,4.033363,12.1,0,0,18.777143,-14.670714,39.134259,-120.953341
1,151,1,1804,Petroleum Liquids,0.000000,71.2,0,0,34.500000,-23.700000,37.629360,-120.931390
3,161,1,1964,Conventional Hydroelectric,0.366669,1.1,0,0,18.777143,-14.670714,37.611461,-120.594678
6,162,1,1360,Conventional Hydroelectric,0.166668,0.5,0,0,18.777143,-14.670714,37.621536,-120.746247
8,180,1,1745,Conventional Hydroelectric,0.333336,1.0,0,0,18.777143,-14.670714,40.451930,-121.862150
9,214,1,744,Conventional Hydroelectric,0.333336,1.0,0,0,18.777143,-14.670714,39.215979,-120.803550
10,215,1,1587,Conventional Hydroelectric,0.466670,1.4,0,0,18.777143,-14.670714,38.071944,-120.539444
11,219,1,8022,Conventional Hydroelectric,39.300290,117.9,0,0,57.100000,-39.300000,40.007445,-121.249438
12,220,H1,2220,Conventional Hydroelectric,11.000081,33.0,0,0,0.000000,0.000000,39.910620,-121.327692
14,221,1,1748,Conventional Hydroelectric,13.333431,40.0,0,0,19.400000,-13.300000,40.175656,-121.190663


In [13]:
CATS_buses = pd.read_csv("GIS/CATS_buses.csv")

In [14]:
CATS_buses.head()

,bus_i,kV,Type,Import,Lat,Lon
0,1,115,'Substation',' ',37.772436,-122.242916
1,2,66,'Substation',' ',37.778482,-121.625951
2,3,66,'Substation',' ',37.748376,-121.677208
3,4,230,'Substation',' ',37.691281,-122.061277
4,5,115,'Substation',' ',37.595782,-122.017435


In [42]:
len(CATS_buses['bus_i'])

8870

In [39]:
graph = collections.defaultdict(list) # from: (to, line)
graph_reverse = collections.defaultdict(list) # to: (from, line)
f = open("GIS/CATS_lines.json")
CATS_lines = json.load(f)
i = 0
line_flow = []
line_to_nodes = []
for feature in CATS_lines['features']:
    line_flow.append(feature['properties']['kV'])
    line_to_nodes.append((feature['properties']['f_bus'], feature['properties']['t_bus']))
    graph[feature['properties']['f_bus']].append((feature['properties']['t_bus'], i))
    graph_reverse[feature['properties']['t_bus']].append((feature['properties']['f_bus'], i))
    i += 1
f.close()
num_lines = len(line_to_nodes)


In [41]:
num_lines

10823

In [40]:
print(line_flow)

[115.0, 115.0, 115.0, 115.0, 115.0, 115.0, 66.0, 66.0, 66.0, 66.0, 66.0, 66.0, 66.0, 66.0, 66.0, 66.0, 66.0, 66.0, 66.0, 66.0, 66.0, 66.0, 66.0, 66.0, 66.0, 66.0, 66.0, 66.0, 66.0, 66.0, 66.0, 66.0, 66.0, 66.0, 66.0, 66.0, 66.0, 66.0, 66.0, 66.0, 66.0, 66.0, 66.0, 66.0, 66.0, 66.0, 66.0, 66.0, 66.0, 66.0, 66.0, 66.0, 66.0, 66.0, 66.0, 66.0, 66.0, 66.0, 66.0, 66.0, 66.0, 66.0, 66.0, 66.0, 66.0, 66.0, 66.0, 66.0, 66.0, 66.0, 66.0, 66.0, 66.0, 66.0, 66.0, 66.0, 66.0, 66.0, 66.0, 66.0, 230.0, 230.0, 230.0, 66.0, 66.0, 66.0, 66.0, 66.0, 66.0, 66.0, 66.0, 66.0, 66.0, 66.0, 66.0, 66.0, 66.0, 66.0, 66.0, 66.0, 66.0, 66.0, 66.0, 66.0, 66.0, 66.0, 66.0, 66.0, 66.0, 66.0, 66.0, 66.0, 66.0, 66.0, 66.0, 66.0, 66.0, 66.0, 66.0, 66.0, 66.0, 66.0, 115.0, 115.0, 115.0, 115.0, 115.0, 115.0, 115.0, 115.0, 115.0, 115.0, 115.0, 115.0, 115.0, 115.0, 230.0, 115.0, 115.0, 115.0, 115.0, 115.0, 230.0, 230.0, 115.0, 115.0, 115.0, 115.0, 115.0, 115.0, 115.0, 115.0, 115.0, 115.0, 115.0, 115.0, 115.0, 115.0, 66.0, 